In [2]:
!pip install pytorch-metric-learning
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 84.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.auto import tqdm
import faiss
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from pytorch_metric_learning import losses, miners, distances, reducers, testers

In [4]:
device = torch.device("cuda")

In [5]:
!nvidia-smi

Tue Jan 23 04:45:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [6]:
df = pd.read_csv("dialysis_proper_encoding (1).csv", encoding="cp949")

In [7]:
df.head()

,질문,답변,소스,카테고리
0,저희 아버지께서 신장병이 있으신데 갑자기 다리가 아프셔서 병원에 가봤더니 통풍성 관...,"신장과 심장, 간 등의 질환이 있으면 전신이 붓게 됩니다. 전신부종은 간단한 소변 ...",대한신장학회,만성신장질환
1,신부전증의 증상이 심각해지기전까지는 거의 없다고 들었습니다. 그렇다면 신부전증의 진...,"정기적인 혈압측정, 신장 기능 검사 및 소변 검사가 중요합니다. 직장을 다니는 경우...",대한신장학회,만성신장질환
2,건강보험공단에서 하는 1차 검진에서 혈뇨가 나타난다는 결과가 나왔습니다. 어떤 검사...,혈뇨의 흔한 원인은 요로의 세균 감염이며 이 경우 소변에 혈뇨 외에도 세균과 백혈구...,대한신장학회,만성신장질환
3,얼마 전에 조직검사를 받고 국소성 분절성 사구체 경화증이라는 진단을 받았습니다. 의...,수축기 혈압을 125 mmHg 미만으로 더 철저히 조절하시고 단백뇨도 줄이는 약제와...,대한신장학회,만성신장질환
4,2년전 임신 때 임신중독증으로 진단 받았고 제왕절개로 분만하였습니다. 이후에도 단백...,신기능이 저하되고 단백뇨가 심하고 고혈압이 동반된 사구체신염 환자에서는 일반적으로 ...,대한신장학회,만성신장질환


In [9]:
questions = df["질문"].values
answers = df["답변"].values
print(len(questions), len(answers))

135 135


Few shot tuning seems necessary

In [8]:
model_name = "monologg/kobigbird-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
question_embedder = AutoModel.from_pretrained(model_name).to(device)
answer_embedder = AutoModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

In [ ]:
question_input_ids, question_attn_masks = [], []
answer_input_ids, answer_attn_masks = [], []

for i in tqdm(range(len(questions)), position=0, leave=True, desc="preprocessing data"):
  encoded_question = tokenizer(questions[i], max_length=512, truncation=True, padding="max_length")
  question_input_ids.append(encoded_question["input_ids"])
  question_attn_masks.append(encoded_question["attention_mask"])

  encoded_answer = tokenizer(answers[i], max_length=512, truncation=True, padding="max_length")
  answer_input_ids.append(encoded_answer["input_ids"])
  answer_attn_masks.append(encoded_answer["attention_mask"])


preprocessing data:   0%|          | 0/135 [00:00<?, ?it/s]

In [ ]:
question_input_ids = torch.tensor(question_input_ids, dtype=int)
question_attn_masks = torch.tensor(question_attn_masks, dtype=int)
answer_input_ids = torch.tensor(answer_input_ids, dtype=int)
answer_attn_masks = torch.tensor(answer_attn_masks, dtype=int)

question_input_ids.shape, question_attn_masks.shape, answer_input_ids.shape, answer_attn_masks.shape

(torch.Size([135, 512]),
 torch.Size([135, 512]),
 torch.Size([135, 512]),
 torch.Size([135, 512]))

In [ ]:
batch_size = 8
train_data = TensorDataset(question_input_ids, question_attn_masks, answer_input_ids, answer_attn_masks)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
epochs = 30
params = list(question_embedder.parameters()) + list(answer_embedder.parameters())
optimizer = torch.optim.AdamW(params, lr=5e-5)
t_total = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.05 * t_total), num_training_steps=t_total)

question_embedder.zero_grad()
answer_embedder.zero_grad()
torch.cuda.empty_cache()


for epoch in tqdm(range(epochs), position=0, leave=True, total=epochs):
  train_loss = 0
  question_embedder.train()
  answer_embedder.train()
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    question_input_ids, question_attn_masks, answer_input_ids, answer_attn_masks = batch
    question_embeddings = question_embedder(question_input_ids, attention_mask=question_attn_masks).pooler_output
    answer_embeddings = answer_embedder(answer_input_ids, attention_mask=answer_attn_masks).pooler_output
    sim_scores = torch.matmul(question_embeddings, torch.transpose(answer_embeddings, 0, 1))
    targets = torch.arange(0, question_embeddings.shape[0]).long().to(device)
    sim_scores = F.log_softmax(sim_scores, dim=1)
    loss = F.nll_loss(sim_scores, targets)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  avg_train_loss = train_loss / len(train_dataloader)
  print(f"epochs: {epoch + 1} | train loss : {train_loss}")

  0%|          | 0/30 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...
Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


epochs: 1 | train loss : 45.370166063308716
epochs: 2 | train loss : 19.31943815946579
epochs: 3 | train loss : 10.995943784713745
epochs: 4 | train loss : 7.995036497712135
epochs: 5 | train loss : 4.295346708968282
epochs: 6 | train loss : 2.4335580356419086
epochs: 7 | train loss : 2.6395758846774697
epochs: 8 | train loss : 3.003965708077885
epochs: 9 | train loss : 2.0500930147245526
epochs: 10 | train loss : 1.511023364495486
epochs: 11 | train loss : 1.3575803868006915
epochs: 12 | train loss : 0.5049916849238798
epochs: 13 | train loss : 0.6702022037934512
epochs: 14 | train loss : 0.5891156528959982
epochs: 15 | train loss : 0.3469881837372668
epochs: 16 | train loss : 0.17385908984579146
epochs: 17 | train loss : 0.08816146565368399
epochs: 18 | train loss : 0.036062532890355214
epochs: 19 | train loss : 0.04248559050029144
epochs: 20 | train loss : 0.03264017988112755
epochs: 21 | train loss : 0.03076562361093238
epochs: 22 | train loss : 0.028428697114577517
epochs: 23 | tr

In [ ]:
# save models
torch.save(question_embedder.state_dict(), "/content/drive/MyDrive/question_embedder.pt")
torch.save(answer_embedder.state_dict(), "/content/drive/MyDrive/answer_embedder.pt")

Inference with checkpoint

In [10]:
question_embedder_chkpt = torch.load("/content/drive/MyDrive/question_embedder.pt")
answer_embedder_chkpt = torch.load("/content/drive/MyDrive/answer_embedder.pt")

In [28]:
model_name = "monologg/kobigbird-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
question_embedder = AutoModel.from_pretrained(model_name).to(device)
question_embedder.load_state_dict(question_embedder_chkpt)
answer_embedder = AutoModel.from_pretrained(model_name).to(device)
answer_embedder.load_state_dict(answer_embedder_chkpt)

<All keys matched successfully>

In [32]:
question_embedder.eval()
answer_embedder.eval()

answer_embeddings = []

question_input_ids, question_attn_masks = [], []
answer_input_ids, answer_attn_masks = [], []

for i in tqdm(range(len(questions)), position=0, leave=True, desc="preprocessing data"):
  encoded_question = tokenizer(questions[i], max_length=512, truncation=True, padding="max_length")
  question_input_ids.append(encoded_question["input_ids"])
  question_attn_masks.append(encoded_question["attention_mask"])

  encoded_answer = tokenizer(answers[i], max_length=512, truncation=True, padding="max_length")
  answer_input_ids.append(encoded_answer["input_ids"])
  answer_attn_masks.append(encoded_answer["attention_mask"])

answer_input_ids = torch.tensor(answer_input_ids, dtype=int)
answer_attn_masks = torch.tensor(answer_attn_masks, dtype=int)

answer_dataset = TensorDataset(answer_input_ids, answer_attn_masks)
answer_dataloader = DataLoader(answer_dataset, batch_size=32)

# get all answer embeddings

for step, batch in tqdm(enumerate(answer_dataloader), total=len(answer_dataloader)):
  batch = tuple(t.to(device) for t in batch)
  answer_input_ids, answer_attn_masks = batch
  with torch.no_grad():
    cur_ans_emb = answer_embedder(answer_input_ids, attention_mask=answer_attn_masks).pooler_output
    cur_ans_emb = cur_ans_emb.detach().cpu()
    for i in range(cur_ans_emb.shape[0]):
      answer_embeddings.append(torch.reshape(cur_ans_emb[i], (-1, 768)))

answer_embeddings = torch.cat(answer_embeddings, dim=0)

preprocessing data:   0%|          | 0/135 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [33]:
answer_embeddings.shape

torch.Size([135, 768])

In [35]:
answer_embeddings = answer_embeddings.to(device)

In [37]:
sample_query = questions[0]
encoded_query = tokenizer(str(sample_query),
                          max_length=512, truncation=True, padding="max_length",
                          return_tensors="pt").to(device)

with torch.no_grad():
  q_emb = question_embedder(**encoded_query).pooler_output
  dot_prod_scores = torch.matmul(q_emb, torch.transpose(answer_embeddings, 0, 1))
  rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

print(f"질문: {sample_query}")
print()
print(f"Rank 1 Answer")
print(answers[rank[0]])
print()
print(f"Rank 2 Answer")
print(answers[rank[1]])
print()
print(f"Rank 3 Answer")
print(answers[rank[2]])

질문: 저희 아버지께서 신장병이 있으신데 갑자기 다리가 아프셔서 병원에 가봤더니 통풍성 관절염이라고 합니다. 아버지께서 다리에 통증이 심하신지 침과 한약을 드신다고 합니다. 전에도 솔잎 엑기스가 몸에 좋다고 하여서 한 달 가량 드시다가 신장이 더 나빠지셨는데 이번에도 한약을 드시다가 부작용이 있을까 봐 걱정이 됩니다.

Rank 1 Answer
신장과 심장, 간 등의 질환이 있으면 전신이 붓게 됩니다. 전신부종은 간단한 소변 및 피검사를 받아보면 쉽게 원인을 알 수 있습니다. 신장에서는 급성 사구체신염, 급성 및 말기 신부전, 특히 신증후군이라는 심한 단백뇨를 나타내는 질환들에서 부종을 볼 수 있습니다. 사구체 신염의 전신 부종은 아침에 일어나면 얼굴이 붓고 일과 후엔 발이 붓습니다. 그러나 여러 검사상 이상이 발견되지 않고도 붓는 경우가 많은데 이를 특발성 부종이라고 부르며 여성에서 흔히 나타납니다. 병원에 가셔서 간단한 검사를 해보세요.

Rank 2 Answer
신장과 심장, 간 등의 질환이 있으면 전신이 붓게 됩니다. 전신부종은 간단한 소변 및 피검사를 받아보면 쉽게 원인을 알 수 있습니다. 신장에서는 급성 사구체신염, 급성 및 말기 신부전, 특히 신증후군이라는 심한 단백뇨를 나타내는 질환들에서 부종을 볼 수 있습니다. 사구체 신염의 전신 부종은 아침에 일어나면 얼굴이 붓고 일과 후엔 발이 붓습니다. 그러나 여러 검사상 이상이 발견되지 않고도 붓는 경우가 많은데 이를 특발성 부종이라고 부르며 여성에서 흔히 나타납니다. 병원에 가셔서 간단한 검사를 해보세요.

Rank 3 Answer
몸이 붓는 것은 신장과 심장, 간 질환 등 여러 가지 원인이 있을 수 있습니다. 전신부종은 소변검사, 혈액검사 등으로 쉽게 원인 파악이 가능한데, 몸이 붓는 증상이 지속될 때는 병원에서 간단한 검사를 받는 것을 권합니다. 신장질환이 생긴 경우 급성 사구체신염, 급성 및 말기신부전, 심한 단백뇨가 있을 경우 부종이 나타나는 경우가 많고, 사구체신염은 아침엔 얼굴이, 일과 후에

In [41]:
sample_query = "몸이 자주 붓는 것 같은데 신장 관련 문제일까요?"

encoded_query = tokenizer(str(sample_query),
                          max_length=512, truncation=True, padding="max_length",
                          return_tensors="pt").to(device)

with torch.no_grad():
  q_emb = question_embedder(**encoded_query).pooler_output
  dot_prod_scores = torch.matmul(q_emb, torch.transpose(answer_embeddings, 0, 1))
  rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

print(f"질문: {sample_query}")
print()
print(f"Rank 1 Answer")
print(answers[rank[0]])
print()
print(f"Rank 2 Answer")
print(answers[rank[1]])
print()
print(f"Rank 3 Answer")
print(answers[rank[2]])

질문: 몸이 자주 붓는 것 같은데 신장 관련 문제일까요?

Rank 1 Answer
신장과 심장, 간 등의 질환이 있으면 전신이 붓게 됩니다. 전신부종은 간단한 소변 및 피검사를 받아보면 쉽게 원인을 알 수 있습니다. 신장에서는 급성 사구체신염, 급성 및 말기 신부전, 특히 신증후군이라는 심한 단백뇨를 나타내는 질환들에서 부종을 볼 수 있습니다. 사구체 신염의 전신 부종은 아침에 일어나면 얼굴이 붓고 일과 후엔 발이 붓습니다. 그러나 여러 검사상 이상이 발견되지 않고도 붓는 경우가 많은데 이를 특발성 부종이라고 부르며 여성에서 흔히 나타납니다. 병원에 가셔서 간단한 검사를 해보세요.

Rank 2 Answer
신장과 심장, 간 등의 질환이 있으면 전신이 붓게 됩니다. 전신부종은 간단한 소변 및 피검사를 받아보면 쉽게 원인을 알 수 있습니다. 신장에서는 급성 사구체신염, 급성 및 말기 신부전, 특히 신증후군이라는 심한 단백뇨를 나타내는 질환들에서 부종을 볼 수 있습니다. 사구체 신염의 전신 부종은 아침에 일어나면 얼굴이 붓고 일과 후엔 발이 붓습니다. 그러나 여러 검사상 이상이 발견되지 않고도 붓는 경우가 많은데 이를 특발성 부종이라고 부르며 여성에서 흔히 나타납니다. 병원에 가셔서 간단한 검사를 해보세요.

Rank 3 Answer
몸이 붓는 것은 신장과 심장, 간 질환 등 여러 가지 원인이 있을 수 있습니다. 전신부종은 소변검사, 혈액검사 등으로 쉽게 원인 파악이 가능한데, 몸이 붓는 증상이 지속될 때는 병원에서 간단한 검사를 받는 것을 권합니다. 신장질환이 생긴 경우 급성 사구체신염, 급성 및 말기신부전, 심한 단백뇨가 있을 경우 부종이 나타나는 경우가 많고, 사구체신염은 아침엔 얼굴이, 일과 후에는 발이 붓는 특징이 나타납니다. 간혹 여성들에게서 검사상 특별한 이상 없이 붓는 경우도 있습니다(특발성 부종).
